In [51]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 6.1.5 Xgboost의 매개변수와 튜닝

## 1. 데이터 불러오기

### 참고자료
- [Titatic dataset 활용한 Xgboost 하이퍼 파라미터 튜닝](https://www.kaggle.com/lifesailor/xgboost/notebook)

In [52]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [61]:
train_file_path = '/content/gdrive/MyDrive/Data Analysis/kaggle/data/titanic/train.csv'
test_file_path = '/content/gdrive/MyDrive/Data Analysis/kaggle/data/titanic/test.csv'

In [62]:
train = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)

In [63]:
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [64]:
print(test.shape)
test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## 2. 데이터 전처리

In [65]:
# 처리하기 복잡한 Column은 제거
del train['Ticket']; del test['Ticket']
del train['Cabin']; del test['Cabin']
del train['Name']; del test['Name']

In [66]:
# train, test에 다른 Category 존재 가능
test.insert(loc=1, column='Survived', value=0)  # 코드를 넣어준 이유는?

In [68]:
# train, test 
total = pd.concat([train, test], axis=0)

In [70]:
# One hot encoding
sex = pd.get_dummies(total['Sex'])
embarked = pd.get_dummies(total['Embarked'])

In [71]:
# 기존 컬럼 제거
del total['Sex']
del total['Embarked']

In [72]:
total = pd.concat([total, sex, embarked], axis=1)
total['Family'] = total['Parch'] + total['SibSp']

In [73]:
# one hot 컬럼이 있는 train, test 
train = total[0:len(train)]
test = total[len(train):]

In [74]:
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,Family
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0,1
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1,0
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1,0


In [75]:
test.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,Family
0,892,0,3,34.5,0,0,7.8292,0,1,0,1,0,0
1,893,0,3,47.0,1,0,7.0000,1,0,0,0,1,1
2,894,0,2,62.0,0,0,9.6875,0,1,0,1,0,0
3,895,0,3,27.0,0,0,8.6625,0,1,0,0,1,0
4,896,0,3,22.0,1,1,12.2875,1,0,0,0,1,2


## 3. 예측 모델 함수 생성

In [77]:
target = 'Survived'
IDcol = 'PassengerId'

In [79]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=100):
   
    # get new n_estimator
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='error', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(alg)
    
    # Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Survived'], eval_metric='error')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Training Accuracy : %.4g" % metrics.accuracy_score(dtrain['Survived'].values, dtrain_predictions))

## 4. 일반적인 Hyperparameter 튜닝 방법
1. high learning rate(0.05 - 0.3)를 선택하고 이 학습률에 맞는 tree 개수를 선정한다. 책에서는 0.1 정도의 약간 큰 값으로 두었다가 세세한 모델 성능을 향상을 위해서는 0.01~0.05로 작게 줄이는 것을 추천한다.

2. tree-specific parameter를 수정한다. (max_depth, min_child_weight, gamma, subsample, colsample_bytree)

4. regularization parameter를 수정한다.

5. 학습률을 낮추고 다시 반복한다.

### 참고자료 
- [[파이썬 머신러닝 완벽 가이드] XGBoost 하이퍼 파라미터 정리](https://dsbook.tistory.com/172)
- [[교재 P.340] PARAMETERS](https://sites.google.com/view/lauraepp/parameters)
- [[교재 P.343~344] Complete Guide to Parameter Tuning in XGBoost with codes in Python](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

### Learning rate와 estimator 수를 고정한다.
초기값은 다음과 같이 선정한다.

1. max_depth = 5: 보통 4-6 를 시작점으로 한다.

2. min_child_weight = 1 : 향후에 튜닝할 것이다.

3. gamma = 0 : 0.1 - 0.2로 시작해도 된다. 그런데 어짜피 튜닝할 것이다.

4. subsample, colsample_bytree = 0.8 : 보통 0.5 - 0.9로 시작한다.

5. scale_pos_weight = 1: Because of high class imbalance.

In [82]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=5,  # Tree 깊이 수
    min_child_weight=1,  
    gamma=0,  # 결정트리를 split하기 위해 최소한으로 줄어햐는 loss 값
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=-1,
    scale_pos_weight=1,
    seed=2019
)
modelfit(xgb1, train, predictors)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=89, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2019,
              silent=None, subsample=0.8, verbosity=1)

Model Report
Training Accuracy : 0.9068


### max_depth와 min_child_weight를 튜닝한다.

In [83]:
param_test1 = {
 'max_depth':range(3,10,3),  # max_depth : 3,6,9
 'min_child_weight':range(1,6,2)  # min_child_weight : 1,3,5
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  nthread=-1, 
                                                  scale_pos_weight=1, seed=2019),
param_grid = param_test1, scoring='accuracy',n_jobs=-1,iid=False, cv=5, verbose=10)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   21.8s finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([0.68301687, 0.64508514, 0.64063148, 1.03520069, 0.9458231 ,
         0.87879896, 1.17142749, 0.99922757, 0.83963957]),
  'mean_score_time': array([0.01314387, 0.01244397, 0.01185179, 0.01927223, 0.01755843,
         0.01532421, 0.02078071, 0.01776881, 0.01494603]),
  'mean_test_score': array([0.81934593, 0.8260875 , 0.83614965, 0.80810997, 0.81259808,
         0.84176135, 0.80473919, 0.81483272, 0.83839684]),
  'param_max_depth': masked_array(data=[3, 3, 3, 6, 6, 6, 9, 9, 9],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_min_child_weight': masked_array(data=[1, 3, 5, 1, 3, 5, 1, 3, 5],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'params': [{'max_depth': 3, 'min_child_weight': 1},
   {'max_depth': 3, 'min_child_weight': 3},


### Gamma를 튜닝한다.

In [84]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]  # gamma 값 : 0 ~ 0.4
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test2, scoring='accuracy', n_jobs=-1, iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([0.65210218, 0.64534311, 0.63520217, 0.63858113, 0.61695189]),
  'mean_score_time': array([0.01335659, 0.01396918, 0.01369114, 0.01214309, 0.01155682]),
  'mean_test_score': array([0.83614965, 0.83278514, 0.8361622 , 0.83840939, 0.83614965]),
  'param_gamma': masked_array(data=[0.0, 0.1, 0.2, 0.3, 0.4],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.1},
   {'gamma': 0.2},
   {'gamma': 0.3},
   {'gamma': 0.4}],
  'rank_test_score': array([3, 5, 2, 1, 3], dtype=int32),
  'split0_test_score': array([0.82681564, 0.82122905, 0.81564246, 0.81564246, 0.82681564]),
  'split1_test_score': array([0.8258427 , 0.80898876, 0.8258427 , 0.82022472, 0.82022472]),
  'split2_test_score': array([0.86516854, 0.85393258, 0.85955056, 0.87078652, 0.86516854]),
  'split3_test_score': array([0.8258427 , 0.83707865, 0.83707865, 0.83707865, 0.83146067]),
  'split4_test_score': arra

### subsample and colsample_bytree를 튜닝한다.

In [85]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],  # sumsample : 0.6 ~ 0.9
 'colsample_bytree':[i/10.0 for i in range(6,10)]  # colsample_bytreee : 0.6 ~ 0.9
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test3, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch3.fit(train[predictors],train[target])
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   26.5s finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([0.63207078, 0.61225033, 0.59750051, 0.59082613, 0.6418448 ,
         0.62956414, 0.62030816, 0.59811621, 0.6722909 , 0.66615238,
         0.63720217, 0.62892957, 0.6975903 , 0.69148521, 0.67418766,
         0.65685024]),
  'mean_score_time': array([0.01323066, 0.01366973, 0.0133647 , 0.01307411, 0.01357274,
         0.01310215, 0.01369052, 0.01413774, 0.01234255, 0.01284728,
         0.01273198, 0.01245623, 0.01260357, 0.01287141, 0.01290121,
         0.01204438]),
  'mean_test_score': array([0.84288494, 0.8361622 , 0.83614965, 0.84063775, 0.8395016 ,
         0.83953926, 0.83616848, 0.84287866, 0.83839684, 0.83616848,
         0.83614965, 0.84176135, 0.83727952, 0.83391501, 0.83279141,
         0.83054422]),
  'param_colsample_bytree': masked_array(data=[0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8,
                     0.8, 0.9, 0.9, 0.9, 0.9],
               mask=[False, False, False, False, False, False, False, False,
                     False, F

### subsample 추가 튜닝하기

In [86]:
param_test4 = {
 'subsample':[i/100.0 for i in range(40,80)],  # subsample : 0.4 ~ 0.79
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.6, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test4, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch4.fit(train[predictors],train[target])
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:   

({'mean_fit_time': array([0.65946054, 0.64522891, 0.62911272, 0.63532696, 0.63639398,
         0.65145659, 0.64921837, 0.65104561, 0.65007119, 0.66097693,
         0.65823164, 0.67084284, 0.65685506, 0.66359406, 0.66789637,
         0.6639173 , 0.6683116 , 0.67224703, 0.66379995, 0.66279116,
         0.66514497, 0.6690289 , 0.66643524, 0.66407752, 0.66187081,
         0.6621335 , 0.66525922, 0.67085414, 0.65909705, 0.66315789,
         0.65126495, 0.65374403, 0.65614033, 0.6507606 , 0.6478888 ,
         0.65571456, 0.6576457 , 0.64710779, 0.64582963, 0.63873701]),
  'mean_score_time': array([0.01234341, 0.01275396, 0.01207848, 0.01490369, 0.01199074,
         0.01236916, 0.01248903, 0.01354685, 0.01257982, 0.01252813,
         0.01251316, 0.0129437 , 0.01280789, 0.01257563, 0.01280451,
         0.01291828, 0.01305971, 0.01320467, 0.01305842, 0.01410842,
         0.01418252, 0.01472464, 0.0126595 , 0.01312318, 0.01297975,
         0.01286564, 0.01299191, 0.01287999, 0.01309476, 0.012861

### Regularization Parameter 튜닝

In [87]:
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.67, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test5, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch5.fit(train[predictors],train[target])
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    8.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    8.0s finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([0.68643203, 0.66904421, 0.6674715 , 0.65801702, 0.37183886]),
  'mean_score_time': array([0.01317825, 0.01405473, 0.01418781, 0.01322808, 0.00428019]),
  'mean_test_score': array([0.8417739 , 0.84514469, 0.84064403, 0.84401481, 0.61616345]),
  'param_reg_alpha': masked_array(data=[1e-05, 0.01, 0.1, 1, 100],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'reg_alpha': 1e-05},
   {'reg_alpha': 0.01},
   {'reg_alpha': 0.1},
   {'reg_alpha': 1},
   {'reg_alpha': 100}],
  'rank_test_score': array([3, 1, 4, 2, 5], dtype=int32),
  'split0_test_score': array([0.82122905, 0.82122905, 0.82681564, 0.82681564, 0.61452514]),
  'split1_test_score': array([0.8258427 , 0.83146067, 0.83146067, 0.83146067, 0.61797753]),
  'split2_test_score': array([0.88202247, 0.88202247, 0.87640449, 0.88202247, 0.61797753]),
  'split3_test_score': array([0.83146067, 0.84269663, 0.83146067, 0.83146067, 0.61797753]),
 

### Learning Rate 감소

In [90]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
    learning_rate =0.01,
    n_estimators=5000,
    max_depth=3,
    min_child_weight=5,
    gamma=0,
    reg_alpha=1e-05,
    subsample=0.67,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=-1,
    scale_pos_weight=1,
    seed=2019
)
modelfit(xgb1, train, predictors)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=None, n_estimators=35, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, seed=2019,
              silent=None, subsample=0.67, verbosity=1)

Model Report
Training Accuracy : 0.8305


# 6.1.6 신경망의 매개변수와 튜닝

## 1. 네트워크 구성
- hidden layer의 활성화 함수 : 'ReLU'가 1순위로 사용되고, 'PReLU', 'LeakyReLU'도 쓰임
- hidden layer의 층수
- 각 layer 유닛 수, Drop-out 비율
- Batch Normalization 적용 여부

## 2. Opitimizer 선택
옵티마이저(Optimizer)는 신경망의 가중치를 어떻게 학습할지에 관한 최적화 알고리즘


- Stochastic gradient descent(확률적 경사하강법), Adam(Adaptive moment estimation)을 주로 사용 (CS231n 7강 참고)
- 어떤 옵티마이저를 사용하더라도 학습률(Learning-rate) 중요한 매개변수가 될 수 있음
- 기타 : batch size 조정, weight decay(가중치 감소)와 같은 정규화 기법 도입

> __learning rate 크기에 따른 학습 loss함수 그래프 (CS231n 7강)__
>
> ![img1](http://aikorea.org/cs231n/assets/nn3/learningrates.jpeg)

## 3. 다층 퍼셉트론의 매개변수 튜닝 예시 (교재 P.346)

### 참고자료
- [[E11] 폐렴아 기다려라 - 우수 노드 선정(이강산님)](https://github.com/RestHope/AIFFEL/blob/main/e11_pneumonia.ipynb)

# 6.1.7 선형 모델의 매개변수와 튜닝
선형 모델은 정규화 매개변수가 튜닝 대상. 튜닝 대상의 매개변수가 적고 계산도 비교적 빠르므로 10배씩 조정한 간격을 범위로 조사할 수 있다.

### 참고자료
- [[G01] 어떤 예측 모형이 차량 수요 예측에 가장 적합할까? (비공개)](https://github.com/AIFFEL-GN-2nd/stevekhkim/blob/main/%5BG02%5D%20Forecasting_Carsharing_Demand.ipynb)

### Ridge

In [ ]:
from sklearn.linear_model import Ridge,RidgeCV,Lasso,LassoCV

In [ ]:
alpha_values_clean = [0.0001, 0.0005, 0.001, 0.002, 0.003, 0.004, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
ridge_op_rate_cv_clean = RidgeCV(alphas=alpha_values_clean, cv=5).fit(x_train, y_train.values.ravel())

print(ridge_op_rate_cv_clean.alpha_)

### Lasso

In [ ]:

alpha_values = [0.0001, 0.0005, 0.001, 0.002, 0.003, 0.004, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
lasso_op_rate_cv = LassoCV(alphas=alpha_values, cv=5).fit(x_train, y_train.values.ravel())

print(lasso_op_rate_cv.alpha_)